In [ ]:
!pip install --quiet qiskit qiskit-ibm-runtime qiskit-aer qiskit-optimization dotenv qiskit-algorithms

## Set up env and backend

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

from qiskit_ibm_runtime import QiskitRuntimeService

api_key = os.getenv("api_key")
crn = os.getenv("crn")

# connect to backend
QiskitRuntimeService.save_account(
    channel="ibm_cloud",
    token=api_key,
    instance=crn,
    overwrite=True
)
# Check that the account has been saved properly
service = QiskitRuntimeService()
service.saved_accounts()

# check backends
service.backends()



[<IBMBackend('ibm_torino')>, <IBMBackend('ibm_brisbane')>]

In [ ]:
from qiskit_aer import Aer
sampler = Sampler(Aer.get_backend("aer_simulator"))


In [ ]:
from qiskit_optimization.applications import Knapsack
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver

prob = Knapsack(values=[3,4,5,6,7], weights=[2,3,4,5,6], max_weight=10)
qp = prob.to_quadratic_program()

print(qp.prettyprint())

In [ ]:
# Numpy Eigensolver
meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
result = meo.solve(qp)
print(result.prettyprint())
print("\nsolution:", prob.interpret(result))

In [ ]:
# from qiskit.primitives import StatevectorSampler

# # QAOA
# meo = MinimumEigenOptimizer(
#     min_eigen_solver=QAOA(reps=1, sampler=StatevectorSampler(seed=123), optimizer=COBYLA())
# )
# result = meo.solve(qp)
# print(result.prettyprint())
# print("\nsolution:", prob.interpret(result))
# print("\ntime:", result.min_eigen_solver_result.optimizer_time)

## Quantum

In [ ]:
def run_knapsack(budget: int, impact: list[int], costs: list[int]) -> list[int]:
    """
    Given a budget and costs and impacts, return the optimal solution in a list using a QAOA algorithm
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_aer.primitives import Sampler
    from qiskit_algorithms.optimizers import COBYLA

    # Build problem
    prob = Knapsack(values=impact, weights=costs, max_weight=budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # Classical baseline
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # Quantum simulator with QAOA
    sampler = Sampler()
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)

    meo_qaoa = MinimumEigenOptimizer(qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print("\nQuantum (QAOA with AerSampler):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    # Return the QAOA solution
    return prob.interpret(result_qaoa)

# run
print("Final:", run_knapsack(10, [1, 2, 3, 4, 5], [4, 5, 6, 7, 8]))


Quadratic Program:
Problem name: Knapsack

Maximize
  x_0 + 2*x_1 + 3*x_2 + 4*x_3 + 5*x_4

Subject to
  Linear constraints (1)
    4*x_0 + 5*x_1 + 6*x_2 + 7*x_3 + 8*x_4 <= 10  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4


Classical (NumPyMinimumEigensolver):
objective function value: 5.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, x_3=0.0, x_4=1.0
status: SUCCESS
solution: [4]


/var/folders/yr/gk8xm7h947qcpzxwlbcp54sm0000gn/T/ipykernel_98421/3209996018.py:38: DeprecationWarning: Sampler has been deprecated as of Aer 0.15, please use SamplerV2 instead.
  print("Final:", run_knapsack(10, [1, 2, 3, 4, 5], [4, 5, 6, 7, 8]))



Quantum (QAOA with AerSampler):
objective function value: 5.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, x_3=0.0, x_4=1.0
status: SUCCESS
solution: [4]
Final: [4]


## Hardware

In [ ]:
def run_knapsack(budget: int, impact: list[int], costs: list[int], backend_name: str = "aer") -> list[int]:
    """
    Solve knapsack with QAOA or classical solver.
    """
    from qiskit_optimization.applications import Knapsack
    from qiskit_optimization.algorithms import MinimumEigenOptimizer
    from qiskit_optimization.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
    from qiskit_algorithms.optimizers import COBYLA

    # --- Build problem ---
    prob = Knapsack(values=impact, weights=costs, max_weight=budget)
    qp = prob.to_quadratic_program()
    print("Quadratic Program:")
    print(qp.prettyprint())

    # --- Classical baseline ---
    meo_classical = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
    result_classical = meo_classical.solve(qp)
    print("\nClassical (NumPyMinimumEigensolver):")
    print(result_classical.prettyprint())
    print("solution:", prob.interpret(result_classical))

    # --- Choose sampler ---
    if backend_name == "aer":
        from qiskit_aer.primitives import Sampler
        sampler = Sampler()
    else:
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
        service = QiskitRuntimeService()
        backend = service.backend(backend_name)
        sampler = Sampler(mode=backend)

    # --- Quantum QAOA ---
    qaoa = QAOA(sampler=sampler, optimizer=COBYLA(maxiter=100), reps=1)
    meo_qaoa = MinimumEigenOptimizer(qaoa)
    result_qaoa = meo_qaoa.solve(qp)
    print(f"\nQuantum (QAOA on {backend_name}):")
    print(result_qaoa.prettyprint())
    print("solution:", prob.interpret(result_qaoa))

    return prob.interpret(result_qaoa)

print("Final:", run_knapsack(10, [1,2,3,4,5], [4,5,6,7,8], backend_name="ibm_torino"))


Quadratic Program:
Problem name: Knapsack

Maximize
  x_0 + 2*x_1 + 3*x_2 + 4*x_3 + 5*x_4

Subject to
  Linear constraints (1)
    4*x_0 + 5*x_1 + 6*x_2 + 7*x_3 + 8*x_4 <= 10  'c0'

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4


Classical (NumPyMinimumEigensolver):
objective function value: 5.0
variable values: x_0=0.0, x_1=0.0, x_2=0.0, x_3=0.0, x_4=1.0
status: SUCCESS
solution: [4]


/Users/abhishekkakolla/code/q3-t-bosons/.venv/lib/python3.11/site-packages/qiskit_optimization/minimum_eigensolvers/qaoa.py:122: UserWarning: Using Sampler V2 (other than StatevectorSampler) without a pass_manager may result in an error. Consider providing a pass_manager for proper circuit transpilation.
  super().__init__(


IBMInputValueError: 'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (-61.47730872913863) outside of this range has been requested; via parameter value(s) γ[0]=-1.921165897785582, substituted in parameter expression 32*γ[0].'